In [2]:
#from google.colab import drive
#mount your drive.  Complete Oauth to authenticate
#drive.mount('/content/gdrive')

In [3]:
#unzip image folder
#!unzip -uq "/content/gdrive/My Drive/jpegs.zip" -d "/content/gdrive/My Drive/"

In [ ]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import numpy as np
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import sys

# Train our network
def train_network(epochs, val_percent, train_batch_size, test_batch_size, eval_freq):
    
    print('Beginning setup...')

    # Training set values (same for all data)
    data_means = [0.6786, 0.6413, 0.6605]
    data_stds = [0.2012, 0.2080, 0.1997]

    transformations = transforms.Compose([
        transforms.Resize((30,40),2),    # Resizes to 1/16 the size of the original image for speed.
    #    transforms.CenterCrop(224),
        transforms.ToTensor(),  # Transforms channels from 0- 255 -> 0-1.
        transforms.Normalize(mean=data_means, std=data_stds)])

    #epochs = 10
    #val_percent = 0.2
    #train_batch_size = 25
    #test_batch_size = 25
    #eval_freq = 1

    # Can choose various loss functions to use:
    criterion = nn.NLLLoss()
    #criterion = nn.CrossEntropyLoss()
    #criterion = nn.MultiMarginLoss(margin=1.0)
    
    print("Setting up train, test, and validation sets...")

    #full_train_set = datasets.ImageFolder("/content/gdrive/My Drive/TRAIN", transform=transformations)
    full_train_set = datasets.ImageFolder("./jpegs/TRAIN", transform=transformations)
    #full_train_set, temp = torch.utils.data.random_split(full_train_set, [int(len(full_train_set) / 20), len(full_train_set) - int(len(full_train_set) / 20)])
    full_train_loader = torch.utils.data.DataLoader(full_train_set, batch_size=train_batch_size, shuffle=True)
    print("Full train set size: ", len(full_train_set))

    val_size = int(len(full_train_set)*val_percent)
    train_set, val_set = torch.utils.data.random_split(full_train_set, [len(full_train_set) - val_size, val_size])
    print("Train set size: ", len(train_set))
    print("Validation set size: ", len(val_set))

    train_loader = torch.utils.data.DataLoader(train_set, batch_size=train_batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_set, batch_size=train_batch_size, shuffle=True)

    #test_set = datasets.ImageFolder("/content/gdrive/My Drive/TEST", transform=transformations)
    test_set = datasets.ImageFolder("./jpegs/TEST", transform=transformations)
    test_loader = torch.utils.data.DataLoader(test_set, batch_size=test_batch_size, shuffle=True)
    print("Test set size: ", len(test_set))

    # Options: MOST GENERALIZED - 121, 169, 201, 161 - MOST ACCURATE
    # https://pytorch.org/hub/pytorch_vision_densenet/
    model = models.densenet161(pretrained=True)


    for param in model.parameters():
        param.requires_grad = False
    classifier_input = model.classifier.in_features
    num_labels = 4
    classifier = nn.Sequential(nn.Linear(classifier_input, 1024),
                               nn.ReLU(),
                               nn.Linear(1024, 512),
                               nn.ReLU(),
                               nn.Linear(512, num_labels),
                               nn.LogSoftmax(dim=1))


    model.classifier = classifier

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model.to(device)
    
    # Set the optimizer function using torch.optim as optim library
    optimizer = optim.Adam(model.classifier.parameters())
    
    print("Beginning training...")

    for epoch in range(epochs):
        train_loss = 0
        val_loss = 0
        accuracy = 0

        # Training the model
        model.train()
        counter = 0
        for inputs, labels in train_loader:
            # Move to device
            inputs, labels = inputs.to(device), labels.to(device)
            # Clear optimizers
            optimizer.zero_grad()
            # Forward pass
            output = model.forward(inputs)
            # Loss
            loss = criterion(output, labels)
            # Calculate gradients (backpropogation)
            loss.backward()
            # Adjust parameters based on gradients
            optimizer.step()
            # Add the loss to the training set's rnning loss
            train_loss += loss.item()*inputs.size(0)

            # Print the progress of our training
            counter += 1
            #print(counter, "/", len(train_loader))
            sys.stdout.write("\rTraining...({}/{})".format(counter, len(train_loader)))
            sys.stdout.flush()

        if (epoch % eval_freq == 0):
    
            print("")

            # Evaluating the model
            model.eval()
            counter = 0
            # Tell torch not to calculate gradients
            with torch.no_grad():
                for inputs, labels in val_loader:
                    # Move to device
                    inputs, labels = inputs.to(device), labels.to(device)
                    # Forward pass
                    output = model.forward(inputs)
                    # Calculate Loss
                    valloss = criterion(output, labels)
                    # Add loss to the validation set's running loss
                    val_loss += valloss.item()*inputs.size(0)

                    # Since our model outputs a LogSoftmax, find the real
                    # percentages by reversing the log function
                    output = torch.exp(output)
                    # Get the top class of the output
                    top_p, top_class = output.topk(1, dim=1)
                    # See how many of the classes were correct?
                    equals = top_class == labels.view(*top_class.shape)
                    # Calculate the mean (get the accuracy for this batch)
                    # and add it to the running accuracy for this epoch
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

                    # Print the progress of our evaluation
                    counter += 1
                    #print(counter, "/", len(val_loader))
                    sys.stdout.write("\rValidating... ({}/{})".format(counter, len(val_loader)))
                    sys.stdout.flush()

            # Get the average loss for the entire epoch
            train_loss = train_loss/len(train_loader.dataset)
            val_loss = val_loss/len(val_loader.dataset)
            accuracy = accuracy/len(val_loader)
            # Print out the information
            print('\nEpoch: {} \tAccuracy: {:.6f} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} '.format(epoch+1, accuracy, train_loss, val_loss))



    full_train_loss = 0
    test_loss = 0
    test_accuracy = 0

    # Training the model one final time on the full dataset
    model.train()
    counter = 0
    for inputs, labels in full_train_loader:
        # Move to device
        inputs, labels = inputs.to(device), labels.to(device)
        # Clear optimizers
        optimizer.zero_grad()
        # Forward pass
        output = model.forward(inputs)
        # Loss
        loss = criterion(output, labels)
        # Calculate gradients (backpropogation)
        loss.backward()
        # Adjust parameters based on gradients
        optimizer.step()
        # Add the loss to the training set's rnning loss
        full_train_loss += loss.item()*inputs.size(0)

        # Print the progress of our training
        counter += 1
        #print(counter, "/", len(full_train_loader))
        sys.stdout.write("\rTraining... ({}/{})".format(counter, len(full_train_loader)))
        sys.stdout.flush()

    # Saving the model
    #torch.save(model, "./blood_model.py")
    torch.save(model, "./blood_model_e{}_v{}_b{}.py".format(epochs, val_percent, train_batch_size))
    
    print("")

    model.eval()
    counter = 0
    # Tell torch not to calculate gradients
    with torch.no_grad():
        for inputs, labels in test_loader:
            # Move to device
            inputs, labels = inputs.to(device), labels.to(device)
            # Forward pass
            output = model.forward(inputs)
            # Calculate Loss
            testloss = criterion(output, labels)
            # Add loss to the validation set's running loss
            test_loss += testloss.item()*inputs.size(0)

            # Since our model outputs a LogSoftmax, find the real
            # percentages by reversing the log function
            output = torch.exp(output)
            # Get the top class of the output
            top_p, top_class = output.topk(1, dim=1)
            # See how many of the classes were correct?
            equals = top_class == labels.view(*top_class.shape)
            # Calculate the mean (get the accuracy for this batch)
            # and add it to the running accuracy for this epoch
            test_accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

            # Print the progress of our evaluation
            counter += 1
            #print(counter, "/", len(test_loader))
            sys.stdout.write("\rTesting...({}/{})".format(counter, len(test_loader)))
            sys.stdout.flush()

    # Get the average loss for the entire fold
    full_train_loss = full_train_loss/len(full_train_loader.dataset)
    test_loss = test_loss/len(test_loader.dataset)
    test_accuracy = test_accuracy/len(test_loader)
    # Print out the information
    #print('Test Set Accuracy: ', test_accuracy)
    print('\nFinal Results:\tAccuracy: {:.6f} \tTraining Loss: {:.6f} \tTesting Loss: {:.6f} \n'.format(test_accuracy, full_train_loss, test_loss))


# Process our image
def process_image(image_path):
    # Load Image
    img = Image.open(image_path)

    # Get the dimensions of the image
    width, height = img.size

    # Resize by keeping the aspect ratio, but changing the dimension
    # so the shortest size is 255px
    # img = img.resize((255, int(255*(height/width))) if width < height else (int(255*(width/height)), 255))

    # Get the dimensions of the new image size
    width, height = img.size

    # Set the coordinates to do a center crop of 224 x 224
    #left = (width - 224)/2
    #top = (height - 224)/2
    #right = (width + 224)/2
    #bottom = (height + 224)/2
    #img = img.crop((left, top, right, bottom))

    # Turn image into numpy array
    img = np.array(img)

    # Make the color channel dimension first instead of last
    img = img.transpose((2, 0, 1))

    # Make all values between 0 and 1
    img = img/255

    # Normalize based on the preset mean and standard deviation
    img[0] = (img[0] - data_means[0])/data_stds[0]
    img[1] = (img[1] - data_means[1])/data_stds[1]
    img[2] = (img[2] - data_means[2])/data_stds[2]

    # Add a fourth dimension to the beginning to indicate batch size
    img = img[np.newaxis,:]

    # Turn into a torch tensor
    image = torch.from_numpy(img)
    image = image.float()
    return image

# Using our model to predict the label
def predict(image, model):
    # Pass the image through our model
    output = model.forward(image)

    # Reverse the log function in our output
    output = torch.exp(output)

    # Get the top predicted class, and the output percentage for
    # that class
    probs, classes = output.topk(1, dim=1)
    return probs.item(), classes.item()

# Show Image
def show_image(image):
    # Convert image to numpy
    image = image.numpy()

    # Un-normalize the image with avg std and mean
    image[0] = image[0] * 0.2030 + 0.6601

    # Print the image
    fig = plt.figure(figsize=(25, 4))
    plt.imshow(np.transpose(image[0], (1, 2, 0)))



In [ ]:
t_epochs = [25,50,100,200,400]
t_batches = [50,100,200,400,800]
t_val = 0.2

for i in t_epochs:
    for j in t_batches:
        print('\nLoss Function: NLLLoss \tNumber of Epochs: {} \tBatch Size: {} \tValidation Percent: {}'.format(i,j,t_val))
        print('Images resized to 30x40px.')
        train_network(epochs=i, val_percent=t_val, train_batch_size=j, test_batch_size=j, eval_freq=1)

#train_network(epochs=3, val_percent=0.2, train_batch_size=25, test_batch_size=25, eval_freq=1)

#epochs = 10
#val_percent = 0.2
#train_batch_size = 25
#test_batch_size = 25
#eval_freq = 1


Loss Function: NLLLoss 	Number of Epochs: 25 	Batch Size: 50 	Validation Percent: 0.2
Images resized to 30x40px.
Beginning setup...
Setting up train, test, and validation sets...
Full train set size:  9957
Train set size:  7966
Validation set size:  1991
Test set size:  2487
Beginning training...
160/160
40/40Epoch: 1 	Accuracy: 0.578183 	Training Loss: 1.142643 	Validation Loss: 1.174141 

Epoch: 2 	Accuracy: 0.611354 	Training Loss: 0.928256 	Validation Loss: 1.119942 

Epoch: 3 	Accuracy: 0.609366 	Training Loss: 0.877490 	Validation Loss: 1.056922 

Epoch: 4 	Accuracy: 0.648524 	Training Loss: 0.810964 	Validation Loss: 0.945965 

Epoch: 5 	Accuracy: 0.663963 	Training Loss: 0.767143 	Validation Loss: 0.851936 

Epoch: 6 	Accuracy: 0.656183 	Training Loss: 0.728475 	Validation Loss: 0.917637 

Epoch: 7 	Accuracy: 0.671244 	Training Loss: 0.658943 	Validation Loss: 0.834392 

Epoch: 8 	Accuracy: 0.671963 	Training Loss: 0.630438 	Validation Loss: 1.083430 

Epoch: 9 	Accuracy: 0.69

In [41]:
'''print(len(full_train_set),len(full_train_set.targets))

# https://deeplizard.com/learn/video/0LhiS6yu2qQ
@torch.no_grad()
def get_all_preds(model, loader):
    all_preds = torch.tensor([])
    for batch in loader:
        images, labels = batch

        preds = model(images)
        all_preds = torch.cat(
            (all_preds, preds)
            ,dim=0
        )
    return all_preds

with torch.no_grad():
    full_train_preds = get_all_preds(model, full_train_loader)
    
preds_correct = get_num_correct(full_train_preds, full_train_set.targets)

print('total correct:', preds_correct)
print('accuracy:', preds_correct / len(full_train_set))'''

9957 9957


RuntimeError: Expected object of device type cuda but got device type cpu for argument #1 'self' in call to _thnn_conv2d_forward

In [6]:
import sys
import time

for i in range(30):
    sys.stdout.write("\r{}>".format("="*i))
    sys.stdout.flush()
    time.sleep(0.5)

=============================>

In [42]:
'''
Run 1

epochs = 10
val_percent = 0.2
train_batch_size = 25
test_batch_size = 25
eval_freq = 1

criterion = nn.NLLLoss()



Epoch: 1 	Accuracy: 0.792156 	Training Loss: 0.866748 	Validation Loss: 0.507381 
Epoch: 2 	Accuracy: 0.828156 	Training Loss: 0.480419 	Validation Loss: 0.436654 
Epoch: 3 	Accuracy: 0.832219 	Training Loss: 0.411089 	Validation Loss: 0.421812 
Epoch: 4 	Accuracy: 0.882937 	Training Loss: 0.364378 	Validation Loss: 0.311798 
Epoch: 5 	Accuracy: 0.884000 	Training Loss: 0.334123 	Validation Loss: 0.299411 
Epoch: 6 	Accuracy: 0.839937 	Training Loss: 0.283902 	Validation Loss: 0.376899 
Epoch: 7 	Accuracy: 0.897500 	Training Loss: 0.245882 	Validation Loss: 0.265919 
Epoch: 8 	Accuracy: 0.845719 	Training Loss: 0.243905 	Validation Loss: 0.433223 
Epoch: 9 	Accuracy: 0.909000 	Training Loss: 0.221514 	Validation Loss: 0.226245 
Epoch: 10 	Accuracy: 0.908156 	Training Loss: 0.224361 	Validation Loss: 0.237138 

Test Set Accuracy:  0.609033337533474
Training Loss: 0.214690 	Testing Loss: 1.398789


Run 2

epochs = 5
val_percent = 0.2
train_batch_size = 25
test_batch_size = 25
eval_freq = 1

criterion = nn.NLLLoss()

Number of Epochs 5 	Batch Size: 25
Full train set size:  9957
Train set size:  7966
Validation set size:  1991
Test set size:  2487
Epoch: 1 	Accuracy: 0.817156 	Training Loss: 0.333088 	Validation Loss: 0.141337 
Epoch: 2 	Accuracy: 0.853875 	Training Loss: 0.156773 	Validation Loss: 0.105850 
Epoch: 3 	Accuracy: 0.820438 	Training Loss: 0.125960 	Validation Loss: 0.134789 
Epoch: 4 	Accuracy: 0.890937 	Training Loss: 0.112367 	Validation Loss: 0.081046 
Epoch: 5 	Accuracy: 0.889656 	Training Loss: 0.090118 	Validation Loss: 0.069279 
Final Results:	Accuracy: 0.633100 	Training Loss: 0.091679 	Testing Loss: 0.378427 


Run 3

epochs = 5
val_percent = 0.2
train_batch_size = 50
test_batch_size = 50
eval_freq = 1

criterion = nn.NLLLoss()

Number of Epochs 5 	Batch Size: 50
Full train set size:  9957
Train set size:  7966
Validation set size:  1991
Test set size:  2487
Epoch: 1 	Accuracy: 0.706902 	Training Loss: 0.338026 	Validation Loss: 0.219773 
Epoch: 2 	Accuracy: 0.845341 	Training Loss: 0.151417 	Validation Loss: 0.110280 
Epoch: 3 	Accuracy: 0.805622 	Training Loss: 0.117722 	Validation Loss: 0.149176 
Epoch: 4 	Accuracy: 0.883841 	Training Loss: 0.100289 	Validation Loss: 0.079365 
Epoch: 5 	Accuracy: 0.869232 	Training Loss: 0.081888 	Validation Loss: 0.091883 
Final Results:	Accuracy: 0.611351 	Training Loss: 0.078182 	Testing Loss: 0.523621 


Run 4

epochs = 10
val_percent = 0.2
train_batch_size = 10
test_batch_size = 10
eval_freq = 1

criterion = nn.MultiMarginLoss(margin=1.0)

Number of Epochs 10 	Batch Size: 10
Full train set size:  9957
Train set size:  7966
Validation set size:  1991
Test set size:  2487
Epoch: 1 	Accuracy: 0.690500 	Training Loss: 0.365573 	Validation Loss: 0.235582 
Epoch: 2 	Accuracy: 0.812500 	Training Loss: 0.201150 	Validation Loss: 0.147381 
Epoch: 3 	Accuracy: 0.730000 	Training Loss: 0.181403 	Validation Loss: 0.185218 
Epoch: 4 	Accuracy: 0.854500 	Training Loss: 0.151453 	Validation Loss: 0.107096 
Epoch: 5 	Accuracy: 0.816000 	Training Loss: 0.146963 	Validation Loss: 0.143466 
Epoch: 6 	Accuracy: 0.904500 	Training Loss: 0.124523 	Validation Loss: 0.063926 
Epoch: 7 	Accuracy: 0.901000 	Training Loss: 0.118210 	Validation Loss: 0.067525 
Epoch: 8 	Accuracy: 0.899500 	Training Loss: 0.106047 	Validation Loss: 0.071042 
Epoch: 9 	Accuracy: 0.882500 	Training Loss: 0.106182 	Validation Loss: 0.090328 
Epoch: 10 	Accuracy: 0.918000 	Training Loss: 0.100359 	Validation Loss: 0.055422 
Final Results:	Accuracy: 0.579575 	Training Loss: 0.099168 	Testing Loss: 0.497934 


Run 5

epochs = 10
val_percent = 0.2
train_batch_size = 25
test_batch_size = 25
eval_freq = 1

criterion = nn.MultiMarginLoss(margin=1.0)

Number of Epochs 10 	Batch Size: 25
Full train set size:  9957
Train set size:  7966
Validation set size:  1991
Test set size:  2487
Epoch: 1 	Accuracy: 0.775875 	Training Loss: 0.369764 	Validation Loss: 0.174198 
Epoch: 2 	Accuracy: 0.864375 	Training Loss: 0.147204 	Validation Loss: 0.094772 
Epoch: 3 	Accuracy: 0.861937 	Training Loss: 0.128951 	Validation Loss: 0.096287 
Epoch: 4 	Accuracy: 0.683875 	Training Loss: 0.115682 	Validation Loss: 0.249141 
Epoch: 5 	Accuracy: 0.864500 	Training Loss: 0.102457 	Validation Loss: 0.099864 
Epoch: 6 	Accuracy: 0.859656 	Training Loss: 0.104447 	Validation Loss: 0.105911 
Epoch: 7 	Accuracy: 0.893437 	Training Loss: 0.084532 	Validation Loss: 0.072831 
Epoch: 8 	Accuracy: 0.891719 	Training Loss: 0.068216 	Validation Loss: 0.078928 
Epoch: 9 	Accuracy: 0.915500 	Training Loss: 0.072132 	Validation Loss: 0.054932 
Epoch: 10 	Accuracy: 0.892437 	Training Loss: 0.064584 	Validation Loss: 0.071456 
Final Results:	Accuracy: 0.591733 	Training Loss: 0.059329 	Testing Loss: 0.465852 


Run 6

epochs = 10
val_percent = 0.2
train_batch_size = 50
test_batch_size = 50
eval_freq = 1

criterion = nn.MultiMarginLoss(margin=1.0)

Number of Epochs 10 	Batch Size: 50
Full train set size:  9957
Train set size:  7966
Validation set size:  1991
Test set size:  2487
Epoch: 1 	Accuracy: 0.809841 	Training Loss: 0.356934 	Validation Loss: 0.159654 
Epoch: 2 	Accuracy: 0.802841 	Training Loss: 0.167494 	Validation Loss: 0.161009 
Epoch: 3 	Accuracy: 0.878280 	Training Loss: 0.128304 	Validation Loss: 0.088309 
Epoch: 4 	Accuracy: 0.898061 	Training Loss: 0.102006 	Validation Loss: 0.071432 
Epoch: 5 	Accuracy: 0.871451 	Training Loss: 0.081752 	Validation Loss: 0.087102 
Epoch: 6 	Accuracy: 0.893671 	Training Loss: 0.073098 	Validation Loss: 0.072326 
Epoch: 7 	Accuracy: 0.831622 	Training Loss: 0.073735 	Validation Loss: 0.115225 
Epoch: 8 	Accuracy: 0.899232 	Training Loss: 0.063430 	Validation Loss: 0.066452 
Epoch: 9 	Accuracy: 0.787622 	Training Loss: 0.057203 	Validation Loss: 0.141842 
Epoch: 10 	Accuracy: 0.906061 	Training Loss: 0.051924 	Validation Loss: 0.061586 
Final Results:	Accuracy: 0.585232 	Training Loss: 0.054685 	Testing Loss: 0.605523 


Run 7

epochs = 25
val_percent = 0.2
train_batch_size = 10
test_batch_size = 10
eval_freq = 1

criterion = nn.MultiMarginLoss(margin=1.0)

Number of Epochs 25 	Batch Size: 10
Full train set size:  9957
Train set size:  7966
Validation set size:  1991
Test set size:  2487
Epoch: 1 	Accuracy: 0.757000 	Training Loss: 0.375817 	Validation Loss: 0.186139 
Epoch: 2 	Accuracy: 0.795000 	Training Loss: 0.205320 	Validation Loss: 0.152541 
Epoch: 3 	Accuracy: 0.880500 	Training Loss: 0.176783 	Validation Loss: 0.086798 
Epoch: 4 	Accuracy: 0.843500 	Training Loss: 0.144934 	Validation Loss: 0.110221 
Epoch: 5 	Accuracy: 0.863500 	Training Loss: 0.142000 	Validation Loss: 0.101107 
Epoch: 6 	Accuracy: 0.884500 	Training Loss: 0.133002 	Validation Loss: 0.080656 
Epoch: 7 	Accuracy: 0.875500 	Training Loss: 0.124930 	Validation Loss: 0.088253 
Epoch: 8 	Accuracy: 0.894500 	Training Loss: 0.120515 	Validation Loss: 0.071422 
Epoch: 9 	Accuracy: 0.910000 	Training Loss: 0.111565 	Validation Loss: 0.064102 
Epoch: 10 	Accuracy: 0.880000 	Training Loss: 0.095075 	Validation Loss: 0.092609 
Epoch: 11 	Accuracy: 0.895500 	Training Loss: 0.092246 	Validation Loss: 0.075103 
Epoch: 12 	Accuracy: 0.877500 	Training Loss: 0.090711 	Validation Loss: 0.086725 
Epoch: 13 	Accuracy: 0.923000 	Training Loss: 0.079825 	Validation Loss: 0.051033 
Epoch: 14 	Accuracy: 0.915500 	Training Loss: 0.079019 	Validation Loss: 0.061295 
Epoch: 15 	Accuracy: 0.875500 	Training Loss: 0.075476 	Validation Loss: 0.085533 
Epoch: 16 	Accuracy: 0.926500 	Training Loss: 0.070708 	Validation Loss: 0.047325 
Epoch: 17 	Accuracy: 0.921500 	Training Loss: 0.067846 	Validation Loss: 0.053646 
Epoch: 18 	Accuracy: 0.929500 	Training Loss: 0.063120 	Validation Loss: 0.049539 
Epoch: 19 	Accuracy: 0.927000 	Training Loss: 0.069536 	Validation Loss: 0.044894 
Epoch: 20 	Accuracy: 0.908000 	Training Loss: 0.057740 	Validation Loss: 0.058151 
Epoch: 21 	Accuracy: 0.931500 	Training Loss: 0.062402 	Validation Loss: 0.044121 
Epoch: 22 	Accuracy: 0.899500 	Training Loss: 0.055306 	Validation Loss: 0.065437 
Epoch: 23 	Accuracy: 0.937500 	Training Loss: 0.053638 	Validation Loss: 0.038308 
Epoch: 24 	Accuracy: 0.877500 	Training Loss: 0.056319 	Validation Loss: 0.089545 
Epoch: 25 	Accuracy: 0.938000 	Training Loss: 0.051920 	Validation Loss: 0.040229 
Final Results:	Accuracy: 0.585083 	Training Loss: 0.057713 	Testing Loss: 0.539717 


Run 8

epochs = 25
val_percent = 0.2
train_batch_size = 25
test_batch_size = 25
eval_freq = 1

criterion = nn.MultiMarginLoss(margin=1.0)

Number of Epochs 25 	Batch Size: 25
Full train set size:  9957
Train set size:  7966
Validation set size:  1991
Test set size:  2487
Epoch: 1 	Accuracy: 0.776719 	Training Loss: 0.350825 	Validation Loss: 0.159253 
Epoch: 2 	Accuracy: 0.852656 	Training Loss: 0.161037 	Validation Loss: 0.111297 
Epoch: 3 	Accuracy: 0.889719 	Training Loss: 0.132416 	Validation Loss: 0.083140 
Epoch: 4 	Accuracy: 0.830875 	Training Loss: 0.113532 	Validation Loss: 0.119885 
Epoch: 5 	Accuracy: 0.894937 	Training Loss: 0.107010 	Validation Loss: 0.078021 
Epoch: 6 	Accuracy: 0.866656 	Training Loss: 0.088975 	Validation Loss: 0.106714 
Epoch: 7 	Accuracy: 0.867937 	Training Loss: 0.092737 	Validation Loss: 0.088912 
Epoch: 8 	Accuracy: 0.916219 	Training Loss: 0.082230 	Validation Loss: 0.060523 
Epoch: 9 	Accuracy: 0.910937 	Training Loss: 0.069651 	Validation Loss: 0.057465 
Epoch: 10 	Accuracy: 0.903937 	Training Loss: 0.061174 	Validation Loss: 0.070802 
Epoch: 11 	Accuracy: 0.921719 	Training Loss: 0.056615 	Validation Loss: 0.054289 
Epoch: 12 	Accuracy: 0.920437 	Training Loss: 0.050735 	Validation Loss: 0.052366 
Epoch: 13 	Accuracy: 0.914437 	Training Loss: 0.055098 	Validation Loss: 0.056898 
Epoch: 14 	Accuracy: 0.934719 	Training Loss: 0.051010 	Validation Loss: 0.045074 
Epoch: 15 	Accuracy: 0.935500 	Training Loss: 0.047064 	Validation Loss: 0.045387 
Epoch: 16 	Accuracy: 0.940937 	Training Loss: 0.048788 	Validation Loss: 0.037858 
Epoch: 17 	Accuracy: 0.920719 	Training Loss: 0.039500 	Validation Loss: 0.057120 
Epoch: 18 	Accuracy: 0.929000 	Training Loss: 0.038879 	Validation Loss: 0.048595 
Epoch: 19 	Accuracy: 0.941437 	Training Loss: 0.039195 	Validation Loss: 0.037387 
Epoch: 20 	Accuracy: 0.938719 	Training Loss: 0.029895 	Validation Loss: 0.043959 
Epoch: 21 	Accuracy: 0.941000 	Training Loss: 0.034057 	Validation Loss: 0.043052 
Epoch: 22 	Accuracy: 0.944719 	Training Loss: 0.027338 	Validation Loss: 0.039243 
Epoch: 23 	Accuracy: 0.940000 	Training Loss: 0.025532 	Validation Loss: 0.045580 
Epoch: 24 	Accuracy: 0.948000 	Training Loss: 0.027400 	Validation Loss: 0.037440 
Epoch: 25 	Accuracy: 0.937219 	Training Loss: 0.026083 	Validation Loss: 0.043362 
Final Results:	Accuracy: 0.564633 	Training Loss: 0.033189 	Testing Loss: 0.646926 


Run 9

epochs = 25
val_percent = 0.2
train_batch_size = 50
test_batch_size = 50
eval_freq = 1

criterion = nn.MultiMarginLoss(margin=1.0)

Number of Epochs 25 	Batch Size: 50
Full train set size:  9957
Train set size:  7966
Validation set size:  1991
Test set size:  2487
Epoch: 1 	Accuracy: 0.835622 	Training Loss: 0.314475 	Validation Loss: 0.121267 
Epoch: 2 	Accuracy: 0.798622 	Training Loss: 0.123585 	Validation Loss: 0.140467 
Epoch: 3 	Accuracy: 0.845622 	Training Loss: 0.115253 	Validation Loss: 0.117120 
Epoch: 4 	Accuracy: 0.895061 	Training Loss: 0.098090 	Validation Loss: 0.079591 
Epoch: 5 	Accuracy: 0.898951 	Training Loss: 0.077107 	Validation Loss: 0.067475 
Epoch: 6 	Accuracy: 0.873341 	Training Loss: 0.078825 	Validation Loss: 0.088468 
Epoch: 7 	Accuracy: 0.890061 	Training Loss: 0.070735 	Validation Loss: 0.075376 
Epoch: 8 	Accuracy: 0.832232 	Training Loss: 0.070780 	Validation Loss: 0.132952 
Epoch: 9 	Accuracy: 0.911390 	Training Loss: 0.068737 	Validation Loss: 0.057574 
Epoch: 10 	Accuracy: 0.906890 	Training Loss: 0.041312 	Validation Loss: 0.069724 
Epoch: 11 	Accuracy: 0.926561 	Training Loss: 0.047419 	Validation Loss: 0.046882 
Epoch: 12 	Accuracy: 0.907561 	Training Loss: 0.033990 	Validation Loss: 0.069589 
Epoch: 13 	Accuracy: 0.936780 	Training Loss: 0.036024 	Validation Loss: 0.042515 
Epoch: 14 	Accuracy: 0.941780 	Training Loss: 0.029332 	Validation Loss: 0.038811 
Epoch: 15 	Accuracy: 0.892671 	Training Loss: 0.037162 	Validation Loss: 0.077334 
Epoch: 16 	Accuracy: 0.939671 	Training Loss: 0.027089 	Validation Loss: 0.041924 
Epoch: 17 	Accuracy: 0.892171 	Training Loss: 0.028560 	Validation Loss: 0.082107 
Epoch: 18 	Accuracy: 0.944280 	Training Loss: 0.033260 	Validation Loss: 0.040751 
Epoch: 19 	Accuracy: 0.941171 	Training Loss: 0.026255 	Validation Loss: 0.039314 
Epoch: 20 	Accuracy: 0.930561 	Training Loss: 0.018340 	Validation Loss: 0.050456 
Epoch: 21 	Accuracy: 0.946561 	Training Loss: 0.020118 	Validation Loss: 0.033873 
Epoch: 22 	Accuracy: 0.926451 	Training Loss: 0.021195 	Validation Loss: 0.056718 
Epoch: 23 	Accuracy: 0.931280 	Training Loss: 0.017675 	Validation Loss: 0.049975 
Epoch: 24 	Accuracy: 0.953890 	Training Loss: 0.024971 	Validation Loss: 0.033525 
Epoch: 25 	Accuracy: 0.949500 	Training Loss: 0.014258 	Validation Loss: 0.033003 
Final Results:	Accuracy: 0.598530 	Training Loss: 0.018827 	Testing Loss: 0.526226 


Run 10

epochs = 50
val_percent = 0.2
train_batch_size = 10
test_batch_size = 10
eval_freq = 1

criterion = nn.MultiMarginLoss(margin=1.0)

Number of Epochs 50 	Batch Size: 10
Full train set size:  9957
Train set size:  7966
Validation set size:  1991
Test set size:  2487
Epoch: 1 	Accuracy: 0.615500 	Training Loss: 0.404495 	Validation Loss: 0.307876 
Epoch: 2 	Accuracy: 0.874000 	Training Loss: 0.204332 	Validation Loss: 0.093994 
Epoch: 3 	Accuracy: 0.878500 	Training Loss: 0.163484 	Validation Loss: 0.092706 
Epoch: 4 	Accuracy: 0.895500 	Training Loss: 0.155909 	Validation Loss: 0.079284 
Epoch: 5 	Accuracy: 0.890000 	Training Loss: 0.135739 	Validation Loss: 0.077842 
Epoch: 6 	Accuracy: 0.865000 	Training Loss: 0.136135 	Validation Loss: 0.101431 
Epoch: 7 	Accuracy: 0.890000 	Training Loss: 0.123805 	Validation Loss: 0.078219 
Epoch: 8 	Accuracy: 0.906500 	Training Loss: 0.111745 	Validation Loss: 0.065994 
Epoch: 9 	Accuracy: 0.919500 	Training Loss: 0.110716 	Validation Loss: 0.057569 
Epoch: 10 	Accuracy: 0.909500 	Training Loss: 0.100474 	Validation Loss: 0.060372 
Epoch: 11 	Accuracy: 0.905000 	Training Loss: 0.105953 	Validation Loss: 0.065719 
Epoch: 12 	Accuracy: 0.925500 	Training Loss: 0.096958 	Validation Loss: 0.050604 
Epoch: 13 	Accuracy: 0.924500 	Training Loss: 0.088848 	Validation Loss: 0.050439 
Epoch: 14 	Accuracy: 0.911000 	Training Loss: 0.082210 	Validation Loss: 0.062055 
Epoch: 15 	Accuracy: 0.916500 	Training Loss: 0.088922 	Validation Loss: 0.056016 
Epoch: 16 	Accuracy: 0.918500 	Training Loss: 0.083045 	Validation Loss: 0.056962 
Epoch: 17 	Accuracy: 0.934500 	Training Loss: 0.076201 	Validation Loss: 0.039789 
Epoch: 18 	Accuracy: 0.928000 	Training Loss: 0.077284 	Validation Loss: 0.045058 
Epoch: 19 	Accuracy: 0.928500 	Training Loss: 0.064828 	Validation Loss: 0.046921 
Epoch: 20 	Accuracy: 0.940000 	Training Loss: 0.069553 	Validation Loss: 0.038839 
Epoch: 21 	Accuracy: 0.917000 	Training Loss: 0.061479 	Validation Loss: 0.059879 
Epoch: 22 	Accuracy: 0.932500 	Training Loss: 0.061752 	Validation Loss: 0.044464 
Epoch: 23 	Accuracy: 0.932500 	Training Loss: 0.067148 	Validation Loss: 0.042130 
Epoch: 24 	Accuracy: 0.887500 	Training Loss: 0.061522 	Validation Loss: 0.069228 
Epoch: 25 	Accuracy: 0.923000 	Training Loss: 0.061900 	Validation Loss: 0.054122 
Epoch: 26 	Accuracy: 0.929500 	Training Loss: 0.058621 	Validation Loss: 0.045503 
Epoch: 27 	Accuracy: 0.940000 	Training Loss: 0.054892 	Validation Loss: 0.042663 
Epoch: 28 	Accuracy: 0.938000 	Training Loss: 0.057282 	Validation Loss: 0.044770 
Epoch: 29 	Accuracy: 0.945500 	Training Loss: 0.056124 	Validation Loss: 0.032680 
Epoch: 30 	Accuracy: 0.935500 	Training Loss: 0.059049 	Validation Loss: 0.043736 
Epoch: 31 	Accuracy: 0.934000 	Training Loss: 0.050999 	Validation Loss: 0.039267 
Epoch: 32 	Accuracy: 0.949000 	Training Loss: 0.050659 	Validation Loss: 0.032414 
Epoch: 33 	Accuracy: 0.942500 	Training Loss: 0.052174 	Validation Loss: 0.034932 
Epoch: 34 	Accuracy: 0.945500 	Training Loss: 0.045395 	Validation Loss: 0.034944 
Epoch: 35 	Accuracy: 0.954000 	Training Loss: 0.049651 	Validation Loss: 0.031507 
Epoch: 36 	Accuracy: 0.945000 	Training Loss: 0.046211 	Validation Loss: 0.032411 
Epoch: 37 	Accuracy: 0.942000 	Training Loss: 0.045516 	Validation Loss: 0.037230 
Epoch: 38 	Accuracy: 0.943000 	Training Loss: 0.049098 	Validation Loss: 0.036136 
Epoch: 39 	Accuracy: 0.946500 	Training Loss: 0.046263 	Validation Loss: 0.030937 
Epoch: 40 	Accuracy: 0.953500 	Training Loss: 0.043737 	Validation Loss: 0.028537 
Epoch: 41 	Accuracy: 0.953500 	Training Loss: 0.044831 	Validation Loss: 0.029608 
Epoch: 42 	Accuracy: 0.947000 	Training Loss: 0.044573 	Validation Loss: 0.032519 
Epoch: 43 	Accuracy: 0.939500 	Training Loss: 0.039212 	Validation Loss: 0.036444 
Epoch: 44 	Accuracy: 0.952500 	Training Loss: 0.042255 	Validation Loss: 0.029741 
Epoch: 45 	Accuracy: 0.950500 	Training Loss: 0.038227 	Validation Loss: 0.031813 
Epoch: 46 	Accuracy: 0.957000 	Training Loss: 0.044166 	Validation Loss: 0.027725 
Epoch: 47 	Accuracy: 0.943500 	Training Loss: 0.034964 	Validation Loss: 0.037995 
Epoch: 48 	Accuracy: 0.933500 	Training Loss: 0.034849 	Validation Loss: 0.047254 
Epoch: 49 	Accuracy: 0.956000 	Training Loss: 0.039633 	Validation Loss: 0.029550 
Epoch: 50 	Accuracy: 0.960500 	Training Loss: 0.036304 	Validation Loss: 0.025657 
Final Results:	Accuracy: 0.551520 	Training Loss: 0.044299 	Testing Loss: 0.742072


Epoch: 9 	Accuracy: 0.861219 	Training Loss: 0.247204 	Validation Loss: 0.344762 
Epoch: 10 	Accuracy: 0.906219 	Training Loss: 0.226018 	Validation Loss: 0.245339 
Final Results:	Accuracy: 0.632933 	Training Loss: 0.231431 	Testing Loss: 1.212736 


Run 11

Loss Function: NLLLoss 	Number of Epochs: 10 	Batch Size: 50 	Validation Percent: 0.2
Full train set size:  9957
Train set size:  7966
Validation set size:  1991
Test set size:  2487
Epoch: 1 	Accuracy: 0.690293 	Training Loss: 0.804683 	Validation Loss: 0.667131 
Epoch: 2 	Accuracy: 0.740683 	Training Loss: 0.486185 	Validation Loss: 0.599333 
Epoch: 3 	Accuracy: 0.900061 	Training Loss: 0.365753 	Validation Loss: 0.272520 
Epoch: 4 	Accuracy: 0.849232 	Training Loss: 0.336812 	Validation Loss: 0.401255 
Epoch: 5 	Accuracy: 0.901561 	Training Loss: 0.302169 	Validation Loss: 0.263551 
Epoch: 6 	Accuracy: 0.916622 	Training Loss: 0.225439 	Validation Loss: 0.210753 
Epoch: 7 	Accuracy: 0.926671 	Training Loss: 0.256873 	Validation Loss: 0.207533 
Epoch: 8 	Accuracy: 0.912451 	Training Loss: 0.197932 	Validation Loss: 0.207827 
Epoch: 9 	Accuracy: 0.904171 	Training Loss: 0.193991 	Validation Loss: 0.225530 
Epoch: 10 	Accuracy: 0.939171 	Training Loss: 0.164434 	Validation Loss: 0.152744 
Final Results:	Accuracy: 0.637492 	Training Loss: 0.158227 	Testing Loss: 1.406197 


Run 12

Loss Function: NLLLoss 	Number of Epochs: 25 	Batch Size: 10 	Validation Percent: 0.2
Full train set size:  9957
Train set size:  7966
Validation set size:  1991
Test set size:  2487
Epoch: 1 	Accuracy: 0.770500 	Training Loss: 0.967307 	Validation Loss: 0.529906 
Epoch: 2 	Accuracy: 0.829500 	Training Loss: 0.609563 	Validation Loss: 0.429458 
Epoch: 3 	Accuracy: 0.879000 	Training Loss: 0.519460 	Validation Loss: 0.316326 
Epoch: 4 	Accuracy: 0.881000 	Training Loss: 0.500834 	Validation Loss: 0.328851 
Epoch: 5 	Accuracy: 0.842000 	Training Loss: 0.441789 	Validation Loss: 0.395934 
Epoch: 6 	Accuracy: 0.894000 	Training Loss: 0.424043 	Validation Loss: 0.284087 
Epoch: 7 	Accuracy: 0.898500 	Training Loss: 0.408934 	Validation Loss: 0.250132 
Epoch: 8 	Accuracy: 0.889500 	Training Loss: 0.386012 	Validation Loss: 0.282248 
Epoch: 9 	Accuracy: 0.897500 	Training Loss: 0.383355 	Validation Loss: 0.267759 
Epoch: 10 	Accuracy: 0.869500 	Training Loss: 0.367704 	Validation Loss: 0.329774 
Epoch: 11 	Accuracy: 0.905500 	Training Loss: 0.340193 	Validation Loss: 0.225654 
Epoch: 12 	Accuracy: 0.913000 	Training Loss: 0.338871 	Validation Loss: 0.216025 
Epoch: 13 	Accuracy: 0.914000 	Training Loss: 0.302225 	Validation Loss: 0.209095 
Epoch: 14 	Accuracy: 0.917000 	Training Loss: 0.320214 	Validation Loss: 0.216257 
Epoch: 15 	Accuracy: 0.931000 	Training Loss: 0.303105 	Validation Loss: 0.190666 
Epoch: 16 	Accuracy: 0.920500 	Training Loss: 0.279768 	Validation Loss: 0.205784 
Epoch: 17 	Accuracy: 0.916000 	Training Loss: 0.285511 	Validation Loss: 0.199706 
Epoch: 18 	Accuracy: 0.919500 	Training Loss: 0.269161 	Validation Loss: 0.209342 
Epoch: 19 	Accuracy: 0.930500 	Training Loss: 0.246930 	Validation Loss: 0.169357 
Epoch: 20 	Accuracy: 0.925000 	Training Loss: 0.260977 	Validation Loss: 0.191430 
Epoch: 21 	Accuracy: 0.928500 	Training Loss: 0.237586 	Validation Loss: 0.184785 
Epoch: 22 	Accuracy: 0.928500 	Training Loss: 0.225773 	Validation Loss: 0.182169 
Epoch: 23 	Accuracy: 0.944000 	Training Loss: 0.216880 	Validation Loss: 0.155428 
Epoch: 24 	Accuracy: 0.930500 	Training Loss: 0.231981 	Validation Loss: 0.177724 
Epoch: 25 	Accuracy: 0.932000 	Training Loss: 0.207603 	Validation Loss: 0.189195 
Final Results:	Accuracy: 0.572060 	Training Loss: 0.223838 	Testing Loss: 1.784645 


Run 13

Loss Function: NLLLoss 	Number of Epochs: 25 	Batch Size: 25 	Validation Percent: 0.2
Full train set size:  9957
Train set size:  7966
Validation set size:  1991
Test set size:  2487
Epoch: 1 	Accuracy: 0.830219 	Training Loss: 0.855608 	Validation Loss: 0.479221 
Epoch: 2 	Accuracy: 0.880875 	Training Loss: 0.498373 	Validation Loss: 0.304472 
Epoch: 3 	Accuracy: 0.832438 	Training Loss: 0.419475 	Validation Loss: 0.399047 
Epoch: 4 	Accuracy: 0.902500 	Training Loss: 0.380038 	Validation Loss: 0.242820 
Epoch: 5 	Accuracy: 0.912500 	Training Loss: 0.299875 	Validation Loss: 0.216459 
Epoch: 6 	Accuracy: 0.865437 	Training Loss: 0.275850 	Validation Loss: 0.311658 
Epoch: 7 	Accuracy: 0.908094 	Training Loss: 0.251369 	Validation Loss: 0.239543 
Epoch: 8 	Accuracy: 0.909719 	Training Loss: 0.237252 	Validation Loss: 0.226684 
Epoch: 9 	Accuracy: 0.930219 	Training Loss: 0.235727 	Validation Loss: 0.188232 
Epoch: 10 	Accuracy: 0.910719 	Training Loss: 0.213302 	Validation Loss: 0.220882 
Epoch: 11 	Accuracy: 0.894156 	Training Loss: 0.194377 	Validation Loss: 0.280342 
Epoch: 12 	Accuracy: 0.941219 	Training Loss: 0.178332 	Validation Loss: 0.155061 
Epoch: 13 	Accuracy: 0.929219 	Training Loss: 0.166174 	Validation Loss: 0.189644 
Epoch: 14 	Accuracy: 0.931719 	Training Loss: 0.167197 	Validation Loss: 0.183463 
Epoch: 15 	Accuracy: 0.929937 	Training Loss: 0.156146 	Validation Loss: 0.185055 
Epoch: 16 	Accuracy: 0.950000 	Training Loss: 0.128333 	Validation Loss: 0.127784 
Epoch: 17 	Accuracy: 0.960000 	Training Loss: 0.098709 	Validation Loss: 0.110066 
Epoch: 18 	Accuracy: 0.936000 	Training Loss: 0.097933 	Validation Loss: 0.185667 
Epoch: 19 	Accuracy: 0.912219 	Training Loss: 0.132238 	Validation Loss: 0.256992 
Epoch: 20 	Accuracy: 0.956500 	Training Loss: 0.104629 	Validation Loss: 0.125040 
Epoch: 21 	Accuracy: 0.922500 	Training Loss: 0.086785 	Validation Loss: 0.228874 
Epoch: 22 	Accuracy: 0.935000 	Training Loss: 0.102344 	Validation Loss: 0.201008 
Epoch: 23 	Accuracy: 0.946000 	Training Loss: 0.115013 	Validation Loss: 0.144343 
Epoch: 24 	Accuracy: 0.956500 	Training Loss: 0.105293 	Validation Loss: 0.116717 
Epoch: 25 	Accuracy: 0.966000 	Training Loss: 0.069673 	Validation Loss: 0.101058 
Final Results:	Accuracy: 0.569433 	Training Loss: 0.110016 	Testing Loss: 2.139700 


Run 14

Loss Function: NLLLoss 	Number of Epochs: 25 	Batch Size: 50 	Validation Percent: 0.2
Full train set size:  9957
Train set size:  7966
Validation set size:  1991
Test set size:  2487
Epoch: 1 	Accuracy: 0.787963 	Training Loss: 0.897462 	Validation Loss: 0.504292 
Epoch: 2 	Accuracy: 0.807232 	Training Loss: 0.491968 	Validation Loss: 0.464330 
Epoch: 3 	Accuracy: 0.862122 	Training Loss: 0.408096 	Validation Loss: 0.328883 
Epoch: 4 	Accuracy: 0.860841 	Training Loss: 0.369210 	Validation Loss: 0.322186 
Epoch: 5 	Accuracy: 0.874951 	Training Loss: 0.321590 	Validation Loss: 0.326853 
Epoch: 6 	Accuracy: 0.911390 	Training Loss: 0.266932 	Validation Loss: 0.213909 
Epoch: 7 	Accuracy: 0.914732 	Training Loss: 0.237886 	Validation Loss: 0.209115 
Epoch: 8 	Accuracy: 0.908780 	Training Loss: 0.207232 	Validation Loss: 0.219961 
Epoch: 9 	Accuracy: 0.904171 	Training Loss: 0.198630 	Validation Loss: 0.229252 
Epoch: 10 	Accuracy: 0.925451 	Training Loss: 0.185391 	Validation Loss: 0.185033 
Epoch: 11 	Accuracy: 0.911561 	Training Loss: 0.177937 	Validation Loss: 0.226930 
Epoch: 12 	Accuracy: 0.935061 	Training Loss: 0.167364 	Validation Loss: 0.160697 
Epoch: 13 	Accuracy: 0.918390 	Training Loss: 0.156361 	Validation Loss: 0.215037 
Epoch: 14 	Accuracy: 0.835780 	Training Loss: 0.168844 	Validation Loss: 0.478164 
Epoch: 15 	Accuracy: 0.927061 	Training Loss: 0.160437 	Validation Loss: 0.178430 
Epoch: 16 	Accuracy: 0.937671 	Training Loss: 0.134645 	Validation Loss: 0.155506 
Epoch: 17 	Accuracy: 0.951890 	Training Loss: 0.078699 	Validation Loss: 0.136037 
Epoch: 18 	Accuracy: 0.913451 	Training Loss: 0.099872 	Validation Loss: 0.259239 
Epoch: 19 	Accuracy: 0.953890 	Training Loss: 0.100969 	Validation Loss: 0.113383 
Epoch: 20 	Accuracy: 0.953671 	Training Loss: 0.113627 	Validation Loss: 0.128499 
Epoch: 21 	Accuracy: 0.927280 	Training Loss: 0.068995 	Validation Loss: 0.203522 
Epoch: 22 	Accuracy: 0.959280 	Training Loss: 0.060184 	Validation Loss: 0.103099 
Epoch: 23 	Accuracy: 0.918890 	Training Loss: 0.060864 	Validation Loss: 0.217166 
Epoch: 24 	Accuracy: 0.954500 	Training Loss: 0.059701 	Validation Loss: 0.131899 
Epoch: 25 	Accuracy: 0.970500 	Training Loss: 0.043710 	Validation Loss: 0.092661 
Final Results:	Accuracy: 0.603589 	Training Loss: 0.092907 	Testing Loss: 1.944045 


Run 15

Loss Function: NLLLoss 	Number of Epochs: 50 	Batch Size: 10 	Validation Percent: 0.2
Full train set size:  9957
Train set size:  7966
Validation set size:  1991
Test set size:  2487
Epoch: 1 	Accuracy: 0.773500 	Training Loss: 0.970494 	Validation Loss: 0.572447 
Epoch: 2 	Accuracy: 0.838000 	Training Loss: 0.606431 	Validation Loss: 0.453523 
Epoch: 3 	Accuracy: 0.867000 	Training Loss: 0.492486 	Validation Loss: 0.340771 
Epoch: 4 	Accuracy: 0.871500 	Training Loss: 0.495966 	Validation Loss: 0.336861 
Epoch: 5 	Accuracy: 0.891000 	Training Loss: 0.453057 	Validation Loss: 0.290129 
Epoch: 6 	Accuracy: 0.885000 	Training Loss: 0.410669 	Validation Loss: 0.292836 
Epoch: 7 	Accuracy: 0.889500 	Training Loss: 0.400224 	Validation Loss: 0.284332 
Epoch: 8 	Accuracy: 0.835500 	Training Loss: 0.382221 	Validation Loss: 0.379654 
Epoch: 9 	Accuracy: 0.908000 	Training Loss: 0.343570 	Validation Loss: 0.245482 
Epoch: 10 	Accuracy: 0.878500 	Training Loss: 0.334101 	Validation Loss: 0.316069 
Epoch: 11 	Accuracy: 0.919500 	Training Loss: 0.323566 	Validation Loss: 0.214422 
Epoch: 12 	Accuracy: 0.911500 	Training Loss: 0.303907 	Validation Loss: 0.225689 
Epoch: 13 	Accuracy: 0.927500 	Training Loss: 0.278901 	Validation Loss: 0.194093 
Epoch: 14 	Accuracy: 0.920000 	Training Loss: 0.276590 	Validation Loss: 0.209097 
Epoch: 15 	Accuracy: 0.937000 	Training Loss: 0.277939 	Validation Loss: 0.177061 
Epoch: 16 	Accuracy: 0.904500 	Training Loss: 0.259435 	Validation Loss: 0.236691 
Epoch: 17 	Accuracy: 0.942500 	Training Loss: 0.260195 	Validation Loss: 0.151711 
Epoch: 18 	Accuracy: 0.920500 	Training Loss: 0.238594 	Validation Loss: 0.188101 
Epoch: 19 	Accuracy: 0.946000 	Training Loss: 0.212026 	Validation Loss: 0.147180 
Epoch: 20 	Accuracy: 0.933000 	Training Loss: 0.209602 	Validation Loss: 0.160090 
Epoch: 21 	Accuracy: 0.917500 	Training Loss: 0.201314 	Validation Loss: 0.201122 
Epoch: 22 	Accuracy: 0.948500 	Training Loss: 0.212906 	Validation Loss: 0.135884 
Epoch: 23 	Accuracy: 0.923000 	Training Loss: 0.191234 	Validation Loss: 0.187037 
Epoch: 24 	Accuracy: 0.928000 	Training Loss: 0.181849 	Validation Loss: 0.188662 
Epoch: 25 	Accuracy: 0.937500 	Training Loss: 0.181295 	Validation Loss: 0.145349 
Epoch: 26 	Accuracy: 0.935500 	Training Loss: 0.173408 	Validation Loss: 0.162939 
Epoch: 27 	Accuracy: 0.945000 	Training Loss: 0.172051 	Validation Loss: 0.148137 
Epoch: 28 	Accuracy: 0.958500 	Training Loss: 0.149575 	Validation Loss: 0.116008 
Epoch: 29 	Accuracy: 0.945500 	Training Loss: 0.156936 	Validation Loss: 0.138169 
Epoch: 30 	Accuracy: 0.948000 	Training Loss: 0.167449 	Validation Loss: 0.137033 
Epoch: 31 	Accuracy: 0.951000 	Training Loss: 0.153111 	Validation Loss: 0.132454 
Epoch: 32 	Accuracy: 0.948500 	Training Loss: 0.151057 	Validation Loss: 0.144834 
Epoch: 33 	Accuracy: 0.942500 	Training Loss: 0.142528 	Validation Loss: 0.170770 
Epoch: 34 	Accuracy: 0.957500 	Training Loss: 0.139631 	Validation Loss: 0.109953 
Epoch: 35 	Accuracy: 0.944500 	Training Loss: 0.146168 	Validation Loss: 0.158937 
Epoch: 36 	Accuracy: 0.949000 	Training Loss: 0.154020 	Validation Loss: 0.133839 
Epoch: 37 	Accuracy: 0.930500 	Training Loss: 0.141423 	Validation Loss: 0.200024 
Epoch: 38 	Accuracy: 0.952000 	Training Loss: 0.127552 	Validation Loss: 0.113707 
Epoch: 39 	Accuracy: 0.949000 	Training Loss: 0.135605 	Validation Loss: 0.145724 
Epoch: 40 	Accuracy: 0.963000 	Training Loss: 0.128991 	Validation Loss: 0.112127 
Epoch: 41 	Accuracy: 0.957000 	Training Loss: 0.129072 	Validation Loss: 0.104938 
Epoch: 42 	Accuracy: 0.965500 	Training Loss: 0.120065 	Validation Loss: 0.093580 
Epoch: 43 	Accuracy: 0.958500 	Training Loss: 0.134817 	Validation Loss: 0.111286 
Epoch: 44 	Accuracy: 0.963500 	Training Loss: 0.109546 	Validation Loss: 0.102473 
Epoch: 45 	Accuracy: 0.969500 	Training Loss: 0.113328 	Validation Loss: 0.080936 
Epoch: 46 	Accuracy: 0.969000 	Training Loss: 0.125886 	Validation Loss: 0.090551 
Epoch: 47 	Accuracy: 0.944000 	Training Loss: 0.107990 	Validation Loss: 0.150715 
Epoch: 48 	Accuracy: 0.950000 	Training Loss: 0.110817 	Validation Loss: 0.163217 
Epoch: 49 	Accuracy: 0.948500 	Training Loss: 0.103674 	Validation Loss: 0.164590 
Epoch: 50 	Accuracy: 0.954000 	Training Loss: 0.114963 	Validation Loss: 0.121440 
Final Results:	Accuracy: 0.553701 	Training Loss: 0.137106 	Testing Loss: 2.250852 


Run 16

Loss Function: NLLLoss 	Number of Epochs: 50 	Batch Size: 25 	Validation Percent: 0.2
Full train set size:  9957
Train set size:  7966
Validation set size:  1991
Test set size:  2487
Epoch: 1 	Accuracy: 0.799000 	Training Loss: 0.907682 	Validation Loss: 0.495177 
Epoch: 2 	Accuracy: 0.759656 	Training Loss: 0.512498 	Validation Loss: 0.597180 
Epoch: 3 	Accuracy: 0.884094 	Training Loss: 0.434474 	Validation Loss: 0.302490 
Epoch: 4 	Accuracy: 0.867156 	Training Loss: 0.369367 	Validation Loss: 0.345526 
Epoch: 5 	Accuracy: 0.859656 	Training Loss: 0.315009 	Validation Loss: 0.373451 
Epoch: 6 	Accuracy: 0.892500 	Training Loss: 0.307191 	Validation Loss: 0.285387 
Epoch: 7 	Accuracy: 0.912437 	Training Loss: 0.260716 	Validation Loss: 0.220679 
Epoch: 8 	Accuracy: 0.912719 	Training Loss: 0.248557 	Validation Loss: 0.213298 
Epoch: 9 	Accuracy: 0.855375 	Training Loss: 0.232020 	Validation Loss: 0.379515 
Epoch: 10 	Accuracy: 0.901656 	Training Loss: 0.216162 	Validation Loss: 0.264605 
Epoch: 11 	Accuracy: 0.915500 	Training Loss: 0.179599 	Validation Loss: 0.208280 
Epoch: 12 	Accuracy: 0.904500 	Training Loss: 0.194271 	Validation Loss: 0.257396 
Epoch: 13 	Accuracy: 0.930937 	Training Loss: 0.172158 	Validation Loss: 0.190657 
Epoch: 14 	Accuracy: 0.890500 	Training Loss: 0.150538 	Validation Loss: 0.318943 
Epoch: 15 	Accuracy: 0.852156 	Training Loss: 0.143470 	Validation Loss: 0.483747 
Epoch: 16 	Accuracy: 0.928719 	Training Loss: 0.154058 	Validation Loss: 0.205602 
Epoch: 17 	Accuracy: 0.935500 	Training Loss: 0.124643 	Validation Loss: 0.170732 
Epoch: 18 	Accuracy: 0.951500 	Training Loss: 0.131361 	Validation Loss: 0.134944 
Epoch: 19 	Accuracy: 0.942719 	Training Loss: 0.114394 	Validation Loss: 0.151227 
Epoch: 20 	Accuracy: 0.952437 	Training Loss: 0.128288 	Validation Loss: 0.129949 
Epoch: 21 	Accuracy: 0.946719 	Training Loss: 0.112846 	Validation Loss: 0.139828 
Epoch: 22 	Accuracy: 0.954437 	Training Loss: 0.099770 	Validation Loss: 0.122431 
Epoch: 23 	Accuracy: 0.951719 	Training Loss: 0.079102 	Validation Loss: 0.158471 
Epoch: 24 	Accuracy: 0.940500 	Training Loss: 0.086558 	Validation Loss: 0.195820 
Epoch: 25 	Accuracy: 0.953719 	Training Loss: 0.078458 	Validation Loss: 0.134334 
Epoch: 26 	Accuracy: 0.955000 	Training Loss: 0.074199 	Validation Loss: 0.121177 
Epoch: 27 	Accuracy: 0.929000 	Training Loss: 0.074563 	Validation Loss: 0.217670 
Epoch: 28 	Accuracy: 0.960000 	Training Loss: 0.065661 	Validation Loss: 0.122243 
Epoch: 29 	Accuracy: 0.955719 	Training Loss: 0.075949 	Validation Loss: 0.120383 
Epoch: 30 	Accuracy: 0.950000 	Training Loss: 0.084585 	Validation Loss: 0.122354 
Epoch: 31 	Accuracy: 0.960500 	Training Loss: 0.069211 	Validation Loss: 0.119626 
Epoch: 32 	Accuracy: 0.952437 	Training Loss: 0.076621 	Validation Loss: 0.124157 
Epoch: 33 	Accuracy: 0.967500 	Training Loss: 0.062922 	Validation Loss: 0.085132 
Epoch: 34 	Accuracy: 0.958719 	Training Loss: 0.057059 	Validation Loss: 0.124251 
Epoch: 35 	Accuracy: 0.957219 	Training Loss: 0.073105 	Validation Loss: 0.132729 
Epoch: 36 	Accuracy: 0.958500 	Training Loss: 0.051005 	Validation Loss: 0.117122 
Epoch: 37 	Accuracy: 0.959000 	Training Loss: 0.059805 	Validation Loss: 0.138598 
Epoch: 38 	Accuracy: 0.962219 	Training Loss: 0.058748 	Validation Loss: 0.108313 
Epoch: 39 	Accuracy: 0.954000 	Training Loss: 0.053703 	Validation Loss: 0.127600 
Epoch: 40 	Accuracy: 0.957719 	Training Loss: 0.074303 	Validation Loss: 0.120411 
Epoch: 41 	Accuracy: 0.966719 	Training Loss: 0.054813 	Validation Loss: 0.097321 
Epoch: 42 	Accuracy: 0.968500 	Training Loss: 0.047670 	Validation Loss: 0.082359 
Epoch: 43 	Accuracy: 0.966500 	Training Loss: 0.054459 	Validation Loss: 0.100934 
Epoch: 44 	Accuracy: 0.968156 	Training Loss: 0.052016 	Validation Loss: 0.096830 
Epoch: 45 	Accuracy: 0.941000 	Training Loss: 0.049038 	Validation Loss: 0.180916 
Epoch: 46 	Accuracy: 0.958656 	Training Loss: 0.055619 	Validation Loss: 0.122170 
Epoch: 47 	Accuracy: 0.966000 	Training Loss: 0.031796 	Validation Loss: 0.109764 
Epoch: 48 	Accuracy: 0.961500 	Training Loss: 0.048712 	Validation Loss: 0.122513 
Epoch: 49 	Accuracy: 0.964219 	Training Loss: 0.050489 	Validation Loss: 0.132052 
Epoch: 50 	Accuracy: 0.964719 	Training Loss: 0.049733 	Validation Loss: 0.097888 
Final Results:	Accuracy: 0.578233 	Training Loss: 0.065914 	Testing Loss: 1.952858 


Run 17

Loss Function: NLLLoss 	Number of Epochs: 10 	Batch Size: 50 	Validation Percent: 0.2
Images resized to 30x40px.
Beginning setup...
Setting up train, test, and validation sets...
Full train set size:  9957
Train set size:  7966
Validation set size:  1991
Test set size:  2487
Beginning training...
Epoch: 1 	Accuracy: 0.600415 	Training Loss: 1.097977 	Validation Loss: 1.160111 
Epoch: 2 	Accuracy: 0.615024 	Training Loss: 0.916336 	Validation Loss: 1.170981 
Epoch: 3 	Accuracy: 0.659744 	Training Loss: 0.853748 	Validation Loss: 1.069735 
Epoch: 4 	Accuracy: 0.612244 	Training Loss: 0.811477 	Validation Loss: 1.334290 
Epoch: 5 	Accuracy: 0.677024 	Training Loss: 0.761510 	Validation Loss: 0.830155 
Epoch: 6 	Accuracy: 0.678963 	Training Loss: 0.710578 	Validation Loss: 0.892294 
Epoch: 7 	Accuracy: 0.675573 	Training Loss: 0.671650 	Validation Loss: 0.891510 
Epoch: 8 	Accuracy: 0.683963 	Training Loss: 0.611281 	Validation Loss: 0.838476 
Epoch: 9 	Accuracy: 0.679354 	Training Loss: 0.581496 	Validation Loss: 0.808214 
Epoch: 10 	Accuracy: 0.690573 	Training Loss: 0.547951 	Validation Loss: 1.292905 
Final Results:	Accuracy: 0.488930 	Training Loss: 0.588135 	Testing Loss: 1.627002 


Run 18

Loss Function: NLLLoss 	Number of Epochs: 10 	Batch Size: 100 	Validation Percent: 0.2
Images resized to 30x40px.
Beginning setup...
Setting up train, test, and validation sets...
Full train set size:  9957
Train set size:  7966
Validation set size:  1991
Test set size:  2487
Beginning training...
Epoch: 1 	Accuracy: 0.555176 	Training Loss: 1.124714 	Validation Loss: 1.049849 
Epoch: 2 	Accuracy: 0.645368 	Training Loss: 0.877542 	Validation Loss: 0.900281 
Epoch: 3 	Accuracy: 0.631522 	Training Loss: 0.824654 	Validation Loss: 0.868768 
Epoch: 4 	Accuracy: 0.667016 	Training Loss: 0.748276 	Validation Loss: 0.851706 
Epoch: 5 	Accuracy: 0.682368 	Training Loss: 0.693535 	Validation Loss: 0.783253 
Epoch: 6 	Accuracy: 0.681868 	Training Loss: 0.632877 	Validation Loss: 0.782845 
Epoch: 7 	Accuracy: 0.683863 	Training Loss: 0.577922 	Validation Loss: 0.807150 
Epoch: 8 	Accuracy: 0.711665 	Training Loss: 0.521608 	Validation Loss: 0.736940 
Epoch: 9 	Accuracy: 0.707764 	Training Loss: 0.454957 	Validation Loss: 0.785636 
Epoch: 10 	Accuracy: 0.719610 	Training Loss: 0.407335 	Validation Loss: 0.788728 
Final Results:	Accuracy: 0.479269 	Training Loss: 0.450259 	Testing Loss: 1.560974 


Run 19

Loss Function: NLLLoss 	Number of Epochs: 10 	Batch Size: 200 	Validation Percent: 0.2
Images resized to 30x40px.
Beginning setup...
Setting up train, test, and validation sets...
Full train set size:  9957
Train set size:  7966
Validation set size:  1991
Test set size:  2487
Beginning training...
Epoch: 1 	Accuracy: 0.599804 	Training Loss: 1.171397 	Validation Loss: 0.973513 
Epoch: 2 	Accuracy: 0.588874 	Training Loss: 0.937229 	Validation Loss: 1.015982 
Epoch: 3 	Accuracy: 0.612280 	Training Loss: 0.814486 	Validation Loss: 0.986740 
Epoch: 4 	Accuracy: 0.636063 	Training Loss: 0.740062 	Validation Loss: 0.893427 
Epoch: 5 	Accuracy: 0.672327 	Training Loss: 0.669452 	Validation Loss: 0.819181 
Epoch: 6 	Accuracy: 0.674398 	Training Loss: 0.575852 	Validation Loss: 0.836685 
Epoch: 7 	Accuracy: 0.697681 	Training Loss: 0.508199 	Validation Loss: 0.825571 
Epoch: 8 	Accuracy: 0.668016 	Training Loss: 0.431571 	Validation Loss: 0.863401 
Epoch: 9 	Accuracy: 0.695110 	Training Loss: 0.375060 	Validation Loss: 0.817083 
Epoch: 10 	Accuracy: 0.676657 	Training Loss: 0.315847 	Validation Loss: 0.896484 
Final Results:	Accuracy: 0.483174 	Training Loss: 0.388844 	Testing Loss: 1.670899 


Run 20

Loss Function: NLLLoss 	Number of Epochs: 10 	Batch Size: 400 	Validation Percent: 0.2
Images resized to 30x40px.
Beginning setup...
Setting up train, test, and validation sets...
Full train set size:  9957
Train set size:  7966
Validation set size:  1991
Test set size:  2487
Beginning training...
Epoch: 1 	Accuracy: 0.587986 	Training Loss: 1.236846 	Validation Loss: 1.055822 
Epoch: 2 	Accuracy: 0.623366 	Training Loss: 0.961489 	Validation Loss: 0.905803 
Epoch: 3 	Accuracy: 0.622854 	Training Loss: 0.848723 	Validation Loss: 0.916030 
Epoch: 4 	Accuracy: 0.663153 	Training Loss: 0.810521 	Validation Loss: 0.839140 
Epoch: 5 	Accuracy: 0.678538 	Training Loss: 0.719327 	Validation Loss: 0.799447 
Epoch: 6 	Accuracy: 0.666027 	Training Loss: 0.662806 	Validation Loss: 0.839970 
Epoch: 7 	Accuracy: 0.673515 	Training Loss: 0.614681 	Validation Loss: 0.834663 
Epoch: 8 	Accuracy: 0.686119 	Training Loss: 0.533959 	Validation Loss: 0.794181 
Epoch: 9 	Accuracy: 0.696665 	Training Loss: 0.458621 	Validation Loss: 0.787878 
Epoch: 10 	Accuracy: 0.700619 	Training Loss: 0.413524 	Validation Loss: 0.756762 
Final Results:	Accuracy: 0.492397 	Training Loss: 0.442663 	Testing Loss: 1.396512 


Loss Function: NLLLoss 	Number of Epochs: 10 	Batch Size: 800 	Validation Percent: 0.2
Images resized to 30x40px.
Beginning setup...
Setting up train, test, and validation sets...
Full train set size:  9957
Train set size:  7966
Validation set size:  1991
Test set size:  2487
Beginning training...
Epoch: 1 	Accuracy: 0.475972 	Training Loss: 1.314685 	Validation Loss: 1.238930 
Epoch: 2 	Accuracy: 0.574277 	Training Loss: 1.098886 	Validation Loss: 1.051863 
Epoch: 3 	Accuracy: 0.589053 	Training Loss: 0.974349 	Validation Loss: 0.978190 
Epoch: 4 	Accuracy: 0.635212 	Training Loss: 0.904811 	Validation Loss: 0.916149 
Epoch: 5 	Accuracy: 0.646122 	Training Loss: 0.843347 	Validation Loss: 0.872921 
Epoch: 6 	Accuracy: 0.644417 	Training Loss: 0.789788 	Validation Loss: 0.870462 
Epoch: 7 	Accuracy: 0.650500 	Training Loss: 0.748503 	Validation Loss: 0.880995 
Epoch: 8 	Accuracy: 0.655006 	Training Loss: 0.694173 	Validation Loss: 0.856772 
Epoch: 9 	Accuracy: 0.650610 	Training Loss: 0.655904 	Validation Loss: 0.895626 
Epoch: 10 	Accuracy: 0.670179 	Training Loss: 0.624158 	Validation Loss: 0.838727 
Final Results:	Accuracy: 0.539670 	Training Loss: 0.630354 	Testing Loss: 1.293276 

'''

''